# This Notebook

In this notebook, we'll be working on a data hackathon hosted by Analytics Vidhya. FinMan is a Financial Distribution company. Over the last 10 years, they have created an offline distribution channel across India. They sell financial products to consumers by hiring agents into their network. The agents are freelancers and get commission when they make a product sale. Managers at FinMan are primarily responsible for recruiting agents. Once a manager has identified a potential applicant, they explain the business opportunity to the agent. Once the applicant consents to the contract, an petition is made to FinMan for the applicant to become an agent. The date on which this application is submitted is known as application_receipt_date. Over the next 3 months, the potential agent has to undergo a 7 day training at the FinMan branch (to learn about Sales processes and various products) and clear a subsequent examination in order to become a FinMan agent.

As is obvious in the above process, there is a significant investment which FinMan makes in identifying, training and recruiting these agents. However, there are a set of agents who do not bring in the expected resultant business. The purpose is thus to develop a classifier to identify from the pool of applicants those who will meet business expectations once hired. In the training data, there is a boolean column with zeroes and ones presenting whether a previously hired agent met the business source expectation called `Business_Sourced(Target)` that will serve as our variable to be predicted. From the problem statement, we have a description of all the columns in both train and test sets:

```
Variable 	                    Definition
ID 	                            Unique Application ID
Office_PIN 	                    PINCODE of Fintro's Offices
Application_Receipt_Date 	    Date of Application
Applicant_City_PIN 	            PINCODE of Applicant Address
Applicant_Gender 	            Applicant's Gender
Applicant_BirthDate 	        Applicant's Birthdate
Applicant_Marital_Status 	    Applicant's Marital Status
Applicant_Occupation 	        Applicant's Occupation
Applicant_Qualification 	    Applicant's Educational Qualification
Manager_DOJ 	                Manager's Date of Joining
Manager_Joining_Designation 	Manager's Joining Designation in Fintro
Manager_Current_Designation 	Manager's Designation at the time of application sourcing
Manager_Grade 	                Manager's Grade in Fintro
Manager_Status 	                Current Employment Status (Probation / Confirmation)
Manager_Gender 	                Manager's Gender
Manager_DoB 	                Manager's Birthdate
Manager_Num_Application 	    No. of Applications sourced in last 3 months by the Manager
Manager_Num_Coded 	            No. of agents recruited by the manager in last 3 months
Manager_Business 	            Amount of business sourced by the manager in last 3 months
Manager_Num_Products 	        Number of products sold by the manager in last 3 months
Manager_Business2 	            Amount of business sourced by the manager in last 3 months excluding business from their Category A advisor
Manager_Num_Products2 	        Number of products sold by the manager in last 3 months excluding business from their Category A advisor
```

Looking at this list, there are some columns that, even if they did coincidentally increase the performance of the identifier, I believe we will want to exclude. Some, we may generate a feature from, before removal, such as seeing if the applicant gender and manager gender are the same, but in isolation it wouldn't make sense to include the following as predictors in an ethical classifier: Applicant_City_PIN, Applicant_Gender, Applicant_BirthDate, Applicant_Marital_Status, Manager_Gender, Manager_DoB.

# EDA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, confusion_matrix, accuracy_score, plot_roc_curve, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from os import path

In [2]:
train_set = pd.read_csv('./data/train_data.csv')
test_set = pd.read_csv('./data/test_data.csv')

In [3]:
train_set.head()

,ID,Office_PIN,Application_Receipt_Date,Applicant_City_PIN,Applicant_Gender,Applicant_BirthDate,Applicant_Marital_Status,Applicant_Occupation,Applicant_Qualification,Manager_DOJ,...,Manager_Status,Manager_Gender,Manager_DoB,Manager_Num_Application,Manager_Num_Coded,Manager_Business,Manager_Num_Products,Manager_Business2,Manager_Num_Products2,Business_Sourced
0,FIN1000001,842001,4/16/2007,844120.0,M,12/19/1971,M,Others,Graduate,11/10/2005,...,Confirmation,M,2/17/1978,2.0,1.0,335249.0,28.0,335249.0,28.0,0
1,FIN1000002,842001,4/16/2007,844111.0,M,2/17/1983,S,Others,Class XII,11/10/2005,...,Confirmation,M,2/17/1978,2.0,1.0,335249.0,28.0,335249.0,28.0,1
2,FIN1000003,800001,4/16/2007,844101.0,M,1/16/1966,M,Business,Class XII,5/27/2006,...,Confirmation,M,3/3/1969,0.0,0.0,357184.0,24.0,357184.0,24.0,0
3,FIN1000004,814112,4/16/2007,814112.0,M,2/3/1988,S,Salaried,Class XII,8/21/2003,...,Confirmation,F,8/14/1978,0.0,0.0,318356.0,22.0,318356.0,22.0,0
4,FIN1000005,814112,4/16/2007,815351.0,M,7/4/1985,M,Others,Class XII,5/8/2006,...,Confirmation,M,2/7/1971,2.0,1.0,230402.0,17.0,230402.0,17.0,0


In [4]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9527 entries, 0 to 9526
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           9527 non-null   object 
 1   Office_PIN                   9527 non-null   int64  
 2   Application_Receipt_Date     9527 non-null   object 
 3   Applicant_City_PIN           9430 non-null   float64
 4   Applicant_Gender             9460 non-null   object 
 5   Applicant_BirthDate          9454 non-null   object 
 6   Applicant_Marital_Status     9454 non-null   object 
 7   Applicant_Occupation         8306 non-null   object 
 8   Applicant_Qualification      9441 non-null   object 
 9   Manager_DOJ                  8844 non-null   object 
 10  Manager_Joining_Designation  8844 non-null   object 
 11  Manager_Current_Designation  8844 non-null   object 
 12  Manager_Grade                8844 non-null   float64
 13  Manager_Status    

In [5]:
train_set.describe(include = ['int64','float64'])

,Office_PIN,Applicant_City_PIN,Manager_Grade,Manager_Num_Application,Manager_Num_Coded,Manager_Business,Manager_Num_Products,Manager_Business2,Manager_Num_Products2,Business_Sourced
count,9527.000000,9430.000000,8844.000000,8844.000000,8844.000000,8.844000e+03,8844.000000,8.844000e+03,8844.000000,9527.00000
mean,452894.372205,456784.547296,3.264134,1.939733,0.758933,1.843710e+05,7.152307,1.829263e+05,7.131275,0.34292
std,235690.618252,239291.081207,1.137449,2.150529,1.188644,2.747163e+05,8.439351,2.718021e+05,8.423597,0.47471
min,110005.000000,110001.000000,1.000000,0.000000,0.000000,-2.652890e+05,0.000000,-2.652890e+05,0.000000,0.00000
25%,226001.000000,226020.000000,2.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.00000
50%,416001.000000,422005.500000,3.000000,1.000000,0.000000,1.021780e+05,5.000000,1.017140e+05,5.000000,0.00000
75%,695014.000000,695017.000000,4.000000,3.000000,1.000000,2.471165e+05,11.000000,2.464612e+05,11.000000,1.00000
max,851101.000000,995657.000000,10.000000,22.000000,9.000000,3.578265e+06,101.000000,3.578265e+06,101.000000,1.00000


In [6]:
train_set.describe(include = ['object'])

,ID,Application_Receipt_Date,Applicant_Gender,Applicant_BirthDate,Applicant_Marital_Status,Applicant_Occupation,Applicant_Qualification,Manager_DOJ,Manager_Joining_Designation,Manager_Current_Designation,Manager_Status,Manager_Gender,Manager_DoB
count,9527,9527,9460,9454,9454,8306,9441,8844,8844,8844,8844,8844,8844
unique,9527,357,2,5836,4,5,11,646,8,5,2,2,1245
top,FIN1004011,5/9/2007,M,1/3/1978,M,Salaried,Class XII,7/9/2007,Level 1,Level 2,Confirmation,M,2/11/1961
freq,1,165,7179,24,6177,3787,5806,106,4632,3208,5277,7627,45


In [7]:
test_set.head()

,ID,Office_PIN,Application_Receipt_Date,Applicant_City_PIN,Applicant_Gender,Applicant_BirthDate,Applicant_Marital_Status,Applicant_Occupation,Applicant_Qualification,Manager_DOJ,...,Manager_Grade,Manager_Status,Manager_Gender,Manager_DoB,Manager_Num_Application,Manager_Num_Coded,Manager_Business,Manager_Num_Products,Manager_Business2,Manager_Num_Products2
0,FIN1009528,284001,7/2/2008,284001.0,M,8/5/1982,S,Salaried,Graduate,5/26/2008,...,2.0,Probation,F,10/14/1978,1.0,0.0,0.0,0.0,0.0,0.0
1,FIN1009529,800001,7/2/2008,800026.0,M,3/11/1968,M,Salaried,Graduate,6/24/2008,...,2.0,Probation,M,3/3/1971,2.0,0.0,0.0,0.0,0.0,0.0
2,FIN1009530,800001,7/2/2008,NaN,M,12/16/1984,S,NaN,Graduate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FIN1009531,800001,7/2/2008,844112.0,M,6/7/1971,S,Salaried,Graduate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FIN1009532,533101,7/2/2008,533102.0,M,7/22/1978,S,NaN,Masters of Business Administration,12/4/2007,...,3.0,Confirmation,M,6/23/1977,4.0,1.0,97352.0,3.0,97352.0,3.0


In [8]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5045 entries, 0 to 5044
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           5045 non-null   object 
 1   Office_PIN                   5045 non-null   int64  
 2   Application_Receipt_Date     5045 non-null   object 
 3   Applicant_City_PIN           5011 non-null   float64
 4   Applicant_Gender             5023 non-null   object 
 5   Applicant_BirthDate          5022 non-null   object 
 6   Applicant_Marital_Status     5015 non-null   object 
 7   Applicant_Occupation         4041 non-null   object 
 8   Applicant_Qualification      5001 non-null   object 
 9   Manager_DOJ                  4221 non-null   object 
 10  Manager_Joining_Designation  4221 non-null   object 
 11  Manager_Current_Designation  4221 non-null   object 
 12  Manager_Grade                4221 non-null   float64
 13  Manager_Status    

In [9]:
test_set.describe(include = ['int64','float64'])

,Office_PIN,Applicant_City_PIN,Manager_Grade,Manager_Num_Application,Manager_Num_Coded,Manager_Business,Manager_Num_Products,Manager_Business2,Manager_Num_Products2
count,5045.000000,5011.000000,4221.000000,4221.000000,4221.000000,4.221000e+03,4221.000000,4.221000e+03,4221.000000
mean,431478.138751,433049.851527,3.352760,2.521204,1.104951,1.115425e+05,3.864250,9.895073e+04,3.495143
std,230425.779147,231655.574011,0.994504,2.491566,1.473985,1.640698e+05,4.688678,1.469193e+05,4.241165
min,110005.000000,110001.000000,2.000000,0.000000,0.000000,-3.196900e+04,0.000000,-3.196900e+04,0.000000
25%,221010.000000,224141.000000,3.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
50%,400705.000000,411005.000000,3.000000,2.000000,1.000000,5.002600e+04,2.000000,4.653700e+04,2.000000
75%,641018.000000,641002.000000,4.000000,4.000000,2.000000,1.757620e+05,7.000000,1.537820e+05,6.000000
max,851101.000000,884112.000000,8.000000,17.000000,10.000000,2.196480e+06,31.000000,2.196480e+06,30.000000


In [10]:
test_set.describe(include = ['object'])

,ID,Application_Receipt_Date,Applicant_Gender,Applicant_BirthDate,Applicant_Marital_Status,Applicant_Occupation,Applicant_Qualification,Manager_DOJ,Manager_Joining_Designation,Manager_Current_Designation,Manager_Status,Manager_Gender,Manager_DoB
count,5045,5045,5023,5022,5015,4041,5001,4221,4221,4221,4221,4221,4221
unique,5045,143,2,3792,4,5,8,465,6,5,2,2,895
top,FIN1011726,7/21/2008,M,1/2/1973,M,Salaried,Graduate,8/4/2008,Level 2,Level 3,Probation,M,5/6/1972
freq,1,273,3900,11,4089,2560,3375,150,1441,1530,2292,3694,32


In [11]:
dates = ['Application_Receipt_Date','Applicant_BirthDate','Manager_DOJ','Manager_DoB']
for column in dates:
    train_set[column] = pd.to_datetime(train_set[column], infer_datetime_format = True)
    test_set[column] = pd.to_datetime(test_set[column], infer_datetime_format = True)

In [12]:
train_set.head()

,ID,Office_PIN,Application_Receipt_Date,Applicant_City_PIN,Applicant_Gender,Applicant_BirthDate,Applicant_Marital_Status,Applicant_Occupation,Applicant_Qualification,Manager_DOJ,...,Manager_Status,Manager_Gender,Manager_DoB,Manager_Num_Application,Manager_Num_Coded,Manager_Business,Manager_Num_Products,Manager_Business2,Manager_Num_Products2,Business_Sourced
0,FIN1000001,842001,2007-04-16,844120.0,M,1971-12-19,M,Others,Graduate,2005-11-10,...,Confirmation,M,1978-02-17,2.0,1.0,335249.0,28.0,335249.0,28.0,0
1,FIN1000002,842001,2007-04-16,844111.0,M,1983-02-17,S,Others,Class XII,2005-11-10,...,Confirmation,M,1978-02-17,2.0,1.0,335249.0,28.0,335249.0,28.0,1
2,FIN1000003,800001,2007-04-16,844101.0,M,1966-01-16,M,Business,Class XII,2006-05-27,...,Confirmation,M,1969-03-03,0.0,0.0,357184.0,24.0,357184.0,24.0,0
3,FIN1000004,814112,2007-04-16,814112.0,M,1988-02-03,S,Salaried,Class XII,2003-08-21,...,Confirmation,F,1978-08-14,0.0,0.0,318356.0,22.0,318356.0,22.0,0
4,FIN1000005,814112,2007-04-16,815351.0,M,1985-07-04,M,Others,Class XII,2006-05-08,...,Confirmation,M,1971-02-07,2.0,1.0,230402.0,17.0,230402.0,17.0,0


In [13]:
test_set.head()

,ID,Office_PIN,Application_Receipt_Date,Applicant_City_PIN,Applicant_Gender,Applicant_BirthDate,Applicant_Marital_Status,Applicant_Occupation,Applicant_Qualification,Manager_DOJ,...,Manager_Grade,Manager_Status,Manager_Gender,Manager_DoB,Manager_Num_Application,Manager_Num_Coded,Manager_Business,Manager_Num_Products,Manager_Business2,Manager_Num_Products2
0,FIN1009528,284001,2008-07-02,284001.0,M,1982-08-05,S,Salaried,Graduate,2008-05-26,...,2.0,Probation,F,1978-10-14,1.0,0.0,0.0,0.0,0.0,0.0
1,FIN1009529,800001,2008-07-02,800026.0,M,1968-03-11,M,Salaried,Graduate,2008-06-24,...,2.0,Probation,M,1971-03-03,2.0,0.0,0.0,0.0,0.0,0.0
2,FIN1009530,800001,2008-07-02,NaN,M,1984-12-16,S,NaN,Graduate,NaT,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,FIN1009531,800001,2008-07-02,844112.0,M,1971-06-07,S,Salaried,Graduate,NaT,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,FIN1009532,533101,2008-07-02,533102.0,M,1978-07-22,S,NaN,Masters of Business Administration,2007-12-04,...,3.0,Confirmation,M,1977-06-23,4.0,1.0,97352.0,3.0,97352.0,3.0


In [14]:
for column in train_set.columns:
    if train_set[column].dtype == 'object' and column != 'ID':
        print('=================================')
        print(f'Column Name: {column}')
        print(train_set[column].value_counts(dropna = False))
        print('=================================')

Column Name: Applicant_Gender
M      7179
F      2281
NaN      67
Name: Applicant_Gender, dtype: int64
Column Name: Applicant_Marital_Status
M      6177
S      3267
NaN      73
W         6
D         4
Name: Applicant_Marital_Status, dtype: int64
Column Name: Applicant_Occupation
Salaried         3787
Business         2303
Others           1966
NaN              1221
Self Employed     149
Student           101
Name: Applicant_Occupation, dtype: int64
Column Name: Applicant_Qualification
Class XII                                                           5806
Graduate                                                            3196
Class X                                                              225
Others                                                               132
NaN                                                                   86
Masters of Business Administration                                    74
Associate / Fellow of Institute of Chartered Accountans of India       3

In [15]:
for column in test_set.columns:
    if test_set[column].dtype == 'object' and column != 'ID':
        print('=================================')
        print(f'Column Name: {column}')
        print(test_set[column].value_counts(dropna = False))
        print('=================================')

Column Name: Applicant_Gender
M      3900
F      1123
NaN      22
Name: Applicant_Gender, dtype: int64
Column Name: Applicant_Marital_Status
M      4089
S       907
NaN      30
W        12
D         7
Name: Applicant_Marital_Status, dtype: int64
Column Name: Applicant_Occupation
Salaried         2560
NaN              1004
Business          646
Others            506
Self Employed     310
Student            19
Name: Applicant_Occupation, dtype: int64
Column Name: Applicant_Qualification
Graduate                                                                              3375
Class XII                                                                             1357
Others                                                                                 171
Masters of Business Administration                                                      71
NaN                                                                                     44
Class X                                                

In columns `Applicant_Qualification` and `Manager_Joining_Designation` there are different unique identifiers. We will have to bin some of them to make the different indentifiers uniform for best performance. We're also seeing significant and annoying NaN counts in these columns, as well as in our numeric columns as is apparent in the df.info()